In [1]:
import time
import json
import logging
import requests
import pandas as pd
from pprint import pprint
from bs4 import BeautifulSoup

In [2]:
LOG_FILENAME = "lastlog.log"
JSON_FILENAME = "ncov_{ctime}.json".format(ctime=time.strftime('%Y%m%d-%H%M%S'))
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
with open(LOG_FILENAME, 'w') as f:
    f.writelines([LOG_FORMAT, '\n'])
logging.basicConfig(filename=LOG_FILENAME, level=logging.DEBUG, format=LOG_FORMAT)

logging.debug("This is a debug log.")
logging.info("This is a info log.")
logging.warning("This is a warning log.")
logging.error("This is a error log.")
logging.critical("This is a critical log.")

In [3]:
response = requests.get("https://ncov.dxy.cn/ncovh5/view/pneumonia")
HTML = response.text.encode(response.encoding).decode()
soup = BeautifulSoup(HTML)
soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>
<html lang="zh-cn" xmlns:layout="http://www.ultraq.net.nz/web/thymeleaf/layout"><head>
<link href="//assets.dxycdn.com/gitrepo/ncov-mobile/dist/umi.bundle.css?t=1580906960587" rel="stylesheet"/>
<meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1,user-scalable=0,viewport-fit=cover" name="viewport"/>
<meta content="#000000" name="theme-color"/>
<title>全国新型肺炎疫情实时动态 - 丁香园·丁香医生</title>
<script>
        window.routerBase = "/ncovh5/view";
    </script>
<script charset="utf-8" src="//assets.dxycdn.com/gitrepo/ncov-mobile/dist/vendors~p__Pneumonia~p__Pneumonia__recommend-list~p__Pneumonia__rumor-list~p__Pneumonia__timeline.async.5ccb549d.js"></script><script charset="utf-8" src="//assets.dxycdn.com/gitrepo/ncov-mobile/dist/vendors~p__Pneumonia~p__Pneumonia__rumor-list.async.9184546f.js"></script><link href="//assets.dxycdn.com/gitrepo/ncov-mobile/dist/vendors~p__Pneumonia.async.9f1a94c3.css" rel="stylesheet" type="text/css"/>

In [4]:
text = soup.contents[1].text
def parse_text(text):
    subtext = text[text.find("window.getAreaStat")::]
    subtext = subtext[subtext.find("[{")::]
    # print(subtext)
    num = 0
    chars = []
    for c in subtext:
        chars.append(c)
        if c == '[':
            num += 1
        if c == ']':
            num -= 1
        if num == 0:
            break
    return ''.join(chars)
counting_json = json.loads(parse_text(text))
print(counting_json)

[{'provinceName': '湖北省', 'provinceShortName': '湖北', 'confirmedCount': 19665, 'suspectedCount': 0, 'curedCount': 633, 'deadCount': 549, 'comment': '', 'locationId': 420000, 'cities': [{'cityName': '武汉', 'confirmedCount': 10117, 'suspectedCount': 0, 'curedCount': 432, 'deadCount': 414, 'locationId': 420100}, {'cityName': '孝感', 'confirmedCount': 1886, 'suspectedCount': 0, 'curedCount': 9, 'deadCount': 25, 'locationId': 420900}, {'cityName': '黄冈', 'confirmedCount': 1807, 'suspectedCount': 0, 'curedCount': 62, 'deadCount': 29, 'locationId': 421100}, {'cityName': '随州', 'confirmedCount': 834, 'suspectedCount': 0, 'curedCount': 9, 'deadCount': 9, 'locationId': 421300}, {'cityName': '荆州', 'confirmedCount': 801, 'suspectedCount': 0, 'curedCount': 18, 'deadCount': 10, 'locationId': 421000}, {'cityName': '襄阳', 'confirmedCount': 787, 'suspectedCount': 0, 'curedCount': 10, 'deadCount': 2, 'locationId': 420600}, {'cityName': '黄石', 'confirmedCount': 566, 'suspectedCount': 0, 'curedCount': 25, 'deadCou

In [5]:
country_df = pd.read_json(json.dumps(counting_json))
country_df.to_json(open(JSON_FILENAME, 'w'))
country_df

,provinceName,provinceShortName,confirmedCount,suspectedCount,curedCount,deadCount,comment,locationId,cities
0,湖北省,湖北,19665,0,633,549,,420000,"[{'cityName': '武汉', 'confirmedCount': 10117, '..."
1,广东省,广东,944,0,49,0,,440000,"[{'cityName': '深圳', 'confirmedCount': 314, 'su..."
2,浙江省,浙江,895,0,78,0,,330000,"[{'cityName': '温州', 'confirmedCount': 364, 'su..."
3,河南省,河南,851,0,54,2,,410000,"[{'cityName': '信阳', 'confirmedCount': 164, 'su..."
4,湖南省,湖南,711,0,56,0,,430000,"[{'cityName': '长沙', 'confirmedCount': 179, 'su..."
5,安徽省,安徽,591,0,23,0,,340000,"[{'cityName': '合肥', 'confirmedCount': 104, 'su..."
6,江西省,江西,548,0,27,0,,360000,"[{'cityName': '南昌', 'confirmedCount': 134, 'su..."
7,重庆市,重庆,389,0,15,2,,500000,"[{'cityName': '万州区', 'confirmedCount': 68, 'su..."
8,江苏省,江苏,373,0,26,0,,320000,"[{'cityName': '苏州', 'confirmedCount': 60, 'sus..."
9,山东省,山东,343,0,18,0,,370000,"[{'cityName': '青岛', 'confirmedCount': 41, 'sus..."


In [6]:
provinces_df = dict()
for idx in country_df.index:
    se = country_df.iloc[idx]
    if se['cities']:
        provinces_df[se['provinceName']] = pd.read_json(json.dumps(se['cities']))
pprint(provinces_df)

{'上海市':    cityName  confirmedCount  suspectedCount  curedCount  deadCount  locationId
0    外地来沪人员              88               0           8          0          -1
1      浦东新区              47               0           3          0      310115
2       宝山区              15               0           0          0      310113
3       静安区              14               0           2          0      310106
4       徐汇区              14               0           0          0      310104
5       闵行区              12               0           1          0      310112
6       长宁区              10               0           0          0      310105
7       松江区              10               0           0          0      310117
8       虹口区               7               0           0          0      310109
9       奉贤区               7               0           0          0      310120
10      杨浦区               6               0           0          0      310110
11      普陀区               6               0 